In [4]:
from dataset.svw import SVW
from tqdm import tqdm

svw = SVW()

videos = svw._get_filtered_videos()

In [5]:
from preprocessing.video import Video
import cv2
import numpy as np


def get_video_as_array(v: Video):
    cap = cv2.VideoCapture(v.video_reshaped_path.as_posix())
    video = [np.zeros((1080, 1920, 3))]
    while (cap.isOpened()):
        ret, img = cap.read()
        if not ret:
            break
        video.append(img)
    cap.release()
    return np.array(video)


In [6]:
import pickle as pkl

with open('/home/rafa/PycharmProjects/sdm2-ready-code/preprocessing/red.pkl', 'rb') as file:
    pca = pkl.load(file)

In [9]:
from preprocessing.dto.detection_dto import DetectionDto
from time import time

progress = tqdm(videos)
skipped = 0
for vid_path in progress:
    vid = Video(vid_path)
    try:
        df = vid.build_detections()
    except:
        continue

    unique_frames = df[DetectionDto.FRAME_ID].unique()
    progress.set_postfix({'path': vid.video_path.as_posix(), 'skipped': skipped, 'frames_num': len(unique_frames)})
    save_path = Video._add_suffix_to_path(vid_path, '_detections_avg_big_ext.pkl')
    if save_path.exists():
        continue

    if df.empty:
        skipped += 1
        with open('/home/rafa/SVW/skipped1.txt', 'a') as file:
            file.write(f'{vid_path.as_posix()} \t - empty \n')
        continue

    if df.shape[0] > 2300:
        skipped += 1
        with open('/home/rafa/SVW/skipped1.txt', 'a') as file:
            file.write(f'{vid_path.as_posix()} \t - long \n')
        continue

    start_time = time()
    # vid_as_arr = get_video_as_array(vid)[unique_frames]
    reduced = pca.transform(np.array(df['visual_features'].to_list()))
    df['visual_features_pca'] = [red for red in reduced]
    progress.set_postfix({'path': vid_path.as_posix(), 'detections_num': df.shape, 'pca_time': time() - start_time})

    # rois = np.array(df['visual_features'].to_list())
    # roi_fun = np.vectorize(get_roi)
    # vid_frames = roi_fun(vid_as_arr[unique_frames])
    #
    # preds = mobilenet.predict(vid_frames, verbose=False)
    # preds_list = [val for val in preds]
    # df['visual_features'] = preds_list
    df.to_pickle(save_path)




  0%|          | 0/3931 [00:38<?, ?it/s, path=/home/rafa/SVW/Videos/archery/423___7cc0d4b318934106b8ab2b026b6aabce.mp4, skipped=0, frames_num=321]


KeyError: 'visual_features'